In [2]:
import os
import copy
import json

from tqdm import tqdm

from valerie.data import load_claims, load_articles, save_claims, save_articles

In [3]:
claims_phase1 = load_claims("data/phase1/all-data/claims.json")
articles_phase1 = load_articles("data/phase1/all-data/articles.json")

all_claims_phase2 = load_claims("data/phase2/all-data/claims.json")
all_articles_phase2 = load_articles("data/phase2/all-data/articles.json")

train_claims_phase2 = load_claims("data/phase2/train-data/claims.json")
train_articles_phase2 = load_articles("data/phase2/train-data/articles.json")

loading articles: 100%|██████████| 50898/50898 [00:00<00:00, 74617.05it/s]


In [4]:
start = max([int(k) for k in claims_phase1.keys()])
start

17141

In [5]:
all_claims_set_phase2 = set(claim.claim for claim in all_claims_phase2.values())
len(all_claims_set_phase2)

13061

In [6]:
unique_claims_phase1 = {}

start = max([int(k) for k in claims_phase1.keys()])*2 + 1
for claim in tqdm(claims_phase1.values()):
    if claim.claim in all_claims_set_phase2:
        continue
    unique_claims_phase1[start] = copy.deepcopy(claim)
    unique_claims_phase1[start].id = start
    start += 1

100%|██████████| 15555/15555 [00:00<00:00, 81687.07it/s]


In [7]:
print(len(claims_phase1))
print(len(unique_claims_phase1))
print(len(claims_phase1) - len(unique_claims_phase1))

15555
6946
8609


In [8]:
unique_articles_phase1 = {}

for k, claim in tqdm(unique_claims_phase1.items()):
    for art_id in claim.related_articles:
        unique_articles_phase1[art_id] = articles_phase1[art_id]

100%|██████████| 6946/6946 [00:00<00:00, 314485.64it/s]


In [9]:
print(len(articles_phase1))
print(len(unique_articles_phase1))
print(len(articles_phase1) - len(unique_articles_phase1))

64974
28619
36355


In [10]:
combined_claims = {**train_claims_phase2, **unique_claims_phase1}
combined_articles = {**train_articles_phase2, **unique_articles_phase1}

In [11]:
print(len(combined_claims))
assert len(set(claim.claim for claim in combined_claims.values())) == len(combined_claims)
assert len(train_claims_phase2) + len(unique_claims_phase1) == len(combined_claims)
print(len(combined_articles))
assert len(train_articles_phase2) + len(unique_articles_phase1) == len(combined_articles)

19353
79517


In [12]:
save_claims(combined_claims, "data/combined/phase1-phase2/claims.json", indent=2)
save_articles(combined_articles, "data/combined/phase1-phase2/articles.json", indent=2)